In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Setup

In [ ]:
import os

!pip -q install accelerate==0.33.0 bitsandbytes==0.43.1 safetensors==0.4.3 tensorboard
!git clone -q https://github.com/kohya-ss/sd-scripts /content/sd-scripts
%cd /content/sd-scripts
!pip -q install -r requirements.txt

# Your dataset made earlier:
driveFolderPath = "/content/drive/MyDrive/training_data/LoRA_Textures"          # dataset matching required images/ and captions/ with .txt
outputFolder  = "/content/drive/MyDrive/outputs/LoRA_v1/"      # where LoRA checkpoints go
logDir  = "/content/drive/MyDrive/outputs/LoRA_v1/log" 
os.makedirs(driveFolderPath, exist_ok=True)
os.makedirs(logDir, exist_ok=True)

!mkdir -p "$outputFolder" "$logDir"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 108.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but yo

In [3]:
import torch, numpy as np, cv2, accelerate, safetensors
print("torch:", torch.__version__)
print("numpy:", np.__version__)
print("opencv:", cv2.__version__)
import diffusers, transformers
print("diffusers:", diffusers.__version__, "transformers:", transformers.__version__)


torch: 2.8.0+cu126
numpy: 2.0.2
opencv: 4.8.1
diffusers: 0.25.0 transformers: 4.44.0


paramaters

In [ ]:
# adjusting for dataset size
numImages = 114          
if numImages <= 60:
    maxSteps = 2500
elif numImages <= 200:
    maxSteps = 3500
else:
    maxSteps = 5000

rank = 8                  # network_dim
lr   = 5e-5               
batch = 4                 
res = "512,512"           # to match tiles

print(f"Training for ~{maxSteps} steps, rank={rank}, LR={lr}, batch={batch}, res={res}")


Training for ~3500 steps, rank=8, LR=5e-05, batch=4, res=512,512


training

In [ ]:
%%bash
# checking if dataset setup properly i.e images and txts together
ls -la "/content/drive/MyDrive/training_data/LoRA_Textures/100_uavtex" | head -n 20

# checking how many of each to be the same (image and txt pairs)
find "/content/drive/MyDrive/training_data/LoRA_Textures/100_uavtex" -type f -iname "*.png" | wc -l
find "/content/drive/MyDrive/training_data/LoRA_Textures/100_uavtex" -type f -iname "*.txt"  | wc -l


total 57874
-rw------- 1 root root 362195 Nov  4 06:59 asphalt_0_0.png
-rw------- 1 root root     74 Nov  4 06:59 asphalt_0_0.txt
-rw------- 1 root root 318034 Nov  4 06:59 asphalt_0_256.png
-rw------- 1 root root     74 Nov  4 06:59 asphalt_0_256.txt
-rw------- 1 root root 455582 Nov  4 06:59 asphalt_1_0_0.png
-rw------- 1 root root     75 Nov  4 06:59 asphalt_1_0_0.txt
-rw------- 1 root root 367734 Nov  4 06:59 asphalt_2_0_0.png
-rw------- 1 root root     74 Nov  4 06:59 asphalt_2_0_0.txt
-rw------- 1 root root 445659 Nov  4 06:59 asphalt_4_0_0.png
-rw------- 1 root root     74 Nov  4 06:59 asphalt_4_0_0.txt
-rw------- 1 root root 467551 Nov  4 06:59 asphalt_5_0_0.png
-rw------- 1 root root     74 Nov  4 06:59 asphalt_5_0_0.txt
-rw------- 1 root root 585771 Nov  4 06:59 asphalt_6_0_0.png
-rw------- 1 root root     74 Nov  4 06:59 asphalt_6_0_0.txt
-rw------- 1 root root 456607 Nov  4 07:00 building_1_0_0.png
-rw------- 1 root root     55 Nov  4 07:00 building_1_0_0.txt
-rw------- 1 r

In [ ]:
!pip install -U "accelerate==0.30.0" "peft==0.10.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [7]:
%%bash
set -e
cd /content/sd-scripts

accelerate launch --mixed_precision=bf16 train_network.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir="/content/drive/MyDrive/training_data/LoRA_Textures" \
  --output_dir="/content/drive/MyDrive/outputs/LoRA_v1/" \
  --logging_dir="/content/drive/MyDrive/outputs/LoRA_v1/log" \
  --resolution=512,512 --enable_bucket \
  --network_module=networks.lora \
  --network_dim=8 --network_alpha=8 \
  --network_train_unet_only \
  --optimizer_type=adamw8bit \
  --learning_rate=5e-5 \
  --lr_scheduler=cosine_with_restarts --lr_warmup_steps=100 \
  --max_train_steps=3500 \
  --save_every_n_steps=500 \
  --train_batch_size=4 \
  --caption_extension=".txt" \
  --shuffle_caption \
  --xformers --persistent_data_loader_workers \
  --cache_latents


accelerator device: cuda


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-11-04 08:23:42.871336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for p

CalledProcessError: Command 'b'set -e\ncd /content/sd-scripts\n\naccelerate launch --mixed_precision=bf16 train_network.py \\\n  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \\\n  --train_data_dir="/content/drive/MyDrive/training_data/LoRA_Textures" \\\n  --output_dir="/content/drive/MyDrive/outputs/LoRA_v1/" \\\n  --logging_dir="/content/drive/MyDrive/outputs/LoRA_v1/log" \\\n  --resolution=512,512 --enable_bucket \\\n  --network_module=networks.lora \\\n  --network_dim=8 --network_alpha=8 \\\n  --network_train_unet_only \\\n  --optimizer_type=adamw8bit \\\n  --learning_rate=5e-5 \\\n  --lr_scheduler=cosine_with_restarts --lr_warmup_steps=100 \\\n  --max_train_steps=3500 \\\n  --save_every_n_steps=500 \\\n  --train_batch_size=4 \\\n  --caption_extension=".txt" \\\n  --shuffle_caption \\\n  --xformers --persistent_data_loader_workers \\\n  --cache_latents\n'' returned non-zero exit status 1.